In [ ]:
# fix random seed for reproducibility
np.random.seed(7)

# load the dataset
symbols = ['^FCHI','^FTSE','^GDAXI','XOM','GE','MSFT','PG','JNJ','^DJI','^IXIC']

data1 = pd.DataFrame()
for sym in symbols:
    data1[sym] = web.DataReader(sym, data_source='yahoo',
        start='1/1/2009', end='4/1/2017')['Adj Close']    # this format for dates needed, other functions may use '01-01-2006'
    data1 = data1.fillna(np.mean(data1.ix[:]))
    data1 =data1.astype('float32')

#Dealing with Missing data 
for i in data1.ix[:]:
    ndata1=np.where(data1==0, np.mean(data1.ix[:]),data1) #the average stock price or average currency price was substituted into the missing data respectively
    rows=[a for a in range(1,(len(data1)+1))]
data1=pd.DataFrame(ndata1,rows,columns=['^FCHI','^FTSE','^GDAXI','XOM','GE','MSFT','PG','JNJ','^DJI','^IXIC'])

data1 = data1.drop(data1.index[[2110,2111,2112,2113,2114]])

# load the dataset
symbols = ['^FCHI','^FTSE','^GDAXI','XOM','GE','MSFT','PG','JNJ','^DJI','^IXIC','SPY']
#symbols = ['SPY']
data = pd.DataFrame()
for sym in symbols:
    data[sym] = web.DataReader(sym, data_source='yahoo',
        start='1/3/2009', end='4/3/2017')['Adj Close']    # this format for dates needed, other functions may use '01-01-2006'
    data = data.fillna(np.mean(data.ix[:]))
    data=data.astype('float32')

#Dealing with Missing data 
for i in data.ix[:]:
    ndata=np.where(data==0, np.mean(data.ix[:]),data) #the average stock price or average currency price was substituted into the missing data respectively
    rows=[a for a in range(1,(len(data)+1))]
data=pd.DataFrame(ndata,rows,columns=['^FCHI','^FTSE','^GDAXI','XOM','GE','MSFT','PG','JNJ','^DJI','^IXIC','SPY'])


#SPY dataset only
dataset = pd.DataFrame(data['SPY'])
dataset = dataset.ix[:,['SPY']].values

# convert an array of values into a dataset matrix at time t,t-1,t-2...
def create_dataset(dataset, timestep):
    dataX, dataY = [], []
    for i in range(len(dataset)-timestep-1):
        a = dataset[i:(i+timestep), 0]
        dataX.append(a)
        dataY.append(dataset[i + timestep, 0])
    return np.array(dataX), np.array(dataY)

#SPY value in the previous three days
dataset = create_dataset(dataset,4)[0]
dataset = pd.DataFrame(dataset)
dataset.columns = ['SPY_t','SPYt-1','SPYt-2','SPYt-3']
#dataset = dataset.drop(dataset.columns[[0]],axis=1)

#concatenate the datasets
features = np.concatenate((dataset,data1), axis=1)
features = pd.DataFrame(features)
features.columns = ['SPY','SPYt-1','SPYt-2','SPYt-3','^FCHI','^FTSE','^GDAXI','XOM','GE','MSFT','PG','JNJ','^DJI','^IXIC']

#saving the data frame as csv file
features.to_csv('features.csv')